In [ ]:
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import fpgrowth, association_rules
import scipy.stats as stats
from statsmodels.stats import multitest as mt

# Read the CSV file into a DataFrame
df = pd.read_csv("C:\\Users\\Jens Harbers\\Documents\\schedenveg_reproduce.csv", sep=" ")

# Manually remove some species based on their abundance in the dataset
# 'n' represents the number of unique areas
n = df.area.nunique()
cnt = df.variable.value_counts() / n

# For those wanting to remove additional species (Big 4 species and some quite abundant ones):
remove_species = ["FesPrat", "LolPere", "PhlPrat", "PoaTriv", "BroErec", "TriFlav", "PlaLanc"]
df = df[~df.variable.isin(remove_species)]

# Get the list of species present in the field "A10_16"
species_on_field = df[df.area == "A10_16"].variable.values

# Remove species from the DataFrame that occur less frequently (below a threshold) in the dataset
df = df[~df.variable.isin(cnt[(cnt <= 0.12)].index.values.tolist())]

# Create a cross-tabulation between 'area' and 'variable' to represent the dataset as a binary matrix
df = pd.crosstab(df.area, df.variable)

# Apply frequent pattern growth algorithm to find frequent itemsets
frequent_itemsets = fpgrowth(df, min_support=0.12, use_colnames=True, max_len=20)

# Apply association rule mining to generate association rules
assoc = association_rules(frequent_itemsets, metric='confidence', min_threshold=0.20, support_only=False)

# Filter the rules: Keep only those where all species in the antecedents are present in the field
assoc["antecedent_in_field"] = assoc.apply(lambda x: x.antecedents.issubset(species_on_field), axis=1)
assoc = assoc[assoc.antecedent_in_field == True]

# Filter the rules further: Keep only those where no species from the 'remove_cons' list is present in the consequents
remove_cons = np.append(species_on_field, ["RumAcet", "DacGlom", "FesPrat", "LolPere", "PhlPrat", "PoaTriv", "PoaPrat"])
assoc["consequent_not_in_field"] = assoc.apply(lambda x: x.consequents.isdisjoint(remove_cons), axis=1)
assoc = assoc[assoc.consequent_not_in_field == True]

# Perform additional calculations on the DataFrame to prepare for statistical tests
assoc["size_antecedents"] = assoc.apply(lambda x: len(x.antecedents), axis=1)
assoc["size_consequents"] = assoc.apply(lambda x: len(x.consequents), axis=1)
assoc['n1x'] = assoc['antecedent support'] * n
assoc['nx1'] = assoc['consequent support'] * n
assoc['n11'] = assoc['support'] * n
assoc['n0x'] = n - assoc['n1x']
assoc['nx0'] = n - assoc['nx1']
assoc['n10'] = assoc["n1x"] - assoc['n11']
assoc['n01'] = assoc["nx1"] - assoc['n11']
assoc['n00'] = assoc['n0x'] - assoc['n01']

# Reduce calculations of Fisher's exact test to speed up the process
df2 = assoc[["n11", "n10", "n01", "n00"]].drop_duplicates().copy()
df2["pval"] = df2.apply(lambda x: stats.fisher_exact([[x.n11, x.n10], [x.n01, x.n00]], "greater")[1], axis=1)
assoc = assoc.merge(df2)

# Perform multiple testing correction on the p-values using various methods
assoc.iloc[:, 9:19] = assoc.iloc[:, 9:19].astype(int)
assoc = assoc.sort_values("pval")
assoc["bonferroni"] = mt.multipletests(assoc.pval, 0.05, method="bonferroni", is_sorted=True)[1]
assoc["sidak"] = mt.multipletests(assoc.pval, 0.05, method="sidak", is_sorted=True)[1]
assoc["holm"] = mt.multipletests(assoc.pval, 0.05, method="holm", is_sorted=True)[1]
assoc["holm_sidak"] = mt.multipletests(assoc.pval, 0.05, method="holm-sidak", is_sorted=True)[1]
assoc["simes_hochberg"] = mt.multipletests(assoc.pval, 0.05, method="simes-hochberg", is_sorted=True)[1]
assoc["fdr_bh"] = mt.multipletests(assoc.pval, 0.05, method="fdr_bh", is_sorted=True)[1]
assoc["fdr_tsbh"] = mt.multipletests(assoc.pval, 0.05, method="fdr_tsbh", is_sorted=True)[1]
assoc["fdr_by"] = mt.multipletests(assoc.pval, 0.05, method="fdr_by", is_sorted=True)[1]
assoc["fdr_tsbky"] = mt.multipletests(assoc.pval, 0.05, method="fdr_tsbky", is_sorted=True)[1]

# Calculate the expected number of species in the consequents based on confidence
assoc["expected_species"] = assoc["size_consequents"] * assoc["confidence"]

# Print the final dimensions of the DataFrame after all calculations
print(f"Datensatzdimension nach allen Berechnungen: {assoc.shape}")

# Print the versions of different libraries being used
import scipy as sc
import statsmodels as st
import mlxtend as ml

print("Version of Pandas: " + pd.__version__)
print("Version of numPy: " + np.__version__)
print("Version of scipy: " + sc.__version__)
print("Version of statsmodels: " + st.__version__)
print("Version of mlxtend: " + ml.__version__)

# Save the results to a CSV file
assoc.to_csv("Saatgutmischungen_10_16_schedenveg_0.12S_0.2C.csv", index=False)

# Select the significant rules based on adjusted p-value and other criteria, and save them to a CSV file
significant = assoc[(assoc.fdr_bh < 0.05) & (assoc.expected_species > 2)].sort_values(
    ["expected_species", "fdr_bh", "size_antecedents"], ascending=[False, False, True]
)
significant = significant.groupby("consequents").first()
significant = significant.sort_values(
    ["expected_species", "fdr_bh", "size_antecedents"], ascending=[False, False, True]
).reset_index()

# Print the number of significant rules after correction
print(f"Anzahl aller signifikanten nach fdr_bh korrigierten Regeln: {significant.shape[0]}")

# Select the top 10 significant rules and perform set union operation on their antecedents and consequents
cp = significant[["antecedents", "consequents", "support", "confidence", "pval", "fdr_bh", "expected_species", "lift"]][:10].copy()
(
    cp.antecedents.values[0] | cp.antecedents.values[1] | cp.antecedents.values[2] | cp.antecedents.values[3] |
    cp.antecedents.values[4] | cp.antecedents.values[5] | cp.antecedents.values[6] | cp.antecedents.values[7] |
    cp.antecedents.values[8] | cp.antecedents.values[9]
)
(
    cp.consequents.values[0] | cp.consequents.values[1] | cp.consequents.values[2] | cp.consequents.values[3] |
    cp.consequents.values[4] | cp.consequents.values[5] | cp.consequents.values[6] | cp.consequents.values[7] |
    cp.consequents.values[8] | cp.consequents.values[9]
)



# List of Abbreviations

| Latin Abbreviation | Full Written Latin Species Name        |
|--------------------|---------------------------------------|
| DacGlom            | Dactylis glomerata                   |
| PlaLanc            | Plantago lanceolata                  |
| MedLupu            | Medicago lupulina                    |
| GalAlbu            | Galium album                         |
| TriPrat            | Trifolium pratensis                  |
| PoaTriv            | Poa trivialis                        |
| TarRude            | Taraxacum sect. Ruderali             |
| LeuIrcu            | Leucanthemum ircutianum              |
| RanBulb            | Ranunculus bulbosus                  |
| AntOdor            | Anthoxanthum odoratum                |
| FesRubr            | Festuca rubra                        |
| CerHolo            | Cerastium holosteoides               |
| CerGlom            | Cerastium glomeratum                 |
| ArrElat            | Arrhenatherum elatius                |
| KnaArve            | Knautia arvensis                     |
| LotCorn            | Lotus corniculatus                   |
| RosSpec            | Rosa specie                          |
| SanMino            | Sanguisorba minor                    |
| TriDubi            | Trifolium dubium                     |
| PriVeri            | Primula veris                        |
| VicAngu            | Vicia angustifolia                   |
| VioHirt            | Viola hirta                          |
| DauCaro            | Daucus carota                        |